# Topic Modelling with Latend Dirichlet Allocation

### Step 1. Load the data

In [5]:
# Loading the dataset from a csv
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

# Total number of documents
print(f'The total number of documents is: {len(documents)}')

C:\Users\User\AppData\Local\Temp\ipykernel_13400\3023278298.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);


The total number of documents is: 300000


In [6]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [10]:
!git push

remote: warning: See http://git.io/iEPt8g for more information.        
remote: warning: File abcnews-date-text.csv is 52.82 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/CalesSla/NLP_Topic_Modelling_LDA.git
   7fa65f6..bd70809  master -> master


### Step 2. Imports and data preprocessing

In [14]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
nltk.download('wordnet')
np.random.seed(400)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# Lemmatizer example
print(WordNetLemmatizer().lemmatize('went', pos = 'v'))

go


In [17]:
# Stemmer example
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [22]:
# Stemming and Lematization on the entire dataset

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [32]:
# Document example after preprocessing

document_num = 0
doc_sample = documents[documents['index'] == 0].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['aba', 'decides', 'against', 'community', 'broadcasting', 'licence']


Tokenized and lemmatized document: 
['decid', 'communiti', 'broadcast', 'licenc']


In [33]:
# Preprocess headlines
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:3]

0     [decid, communiti, broadcast, licenc]
1                        [wit, awar, defam]
2    [call, infrastructur, protect, summit]
Name: headline_text, dtype: object

### Step 3. Bag of Words approach on the dataset

In [34]:
# Create a dictionary of word counts
dictionary = gensim.corpora.Dictionary(processed_docs)

In [35]:
# Remove too rare and too common words from the dictionary
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [36]:
# Initialize the bag of words corpus using the word counts
bow_corpus = [dictionary.doc2bow(document) for document in processed_docs]
bow_corpus[document_num]

[(0, 1), (1, 1), (2, 1), (3, 1)]

In [38]:
# Bag of words document example
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 0 ("broadcast") appears 1 time.
Word 1 ("communiti") appears 1 time.
Word 2 ("decid") appears 1 time.
Word 3 ("licenc") appears 1 time.


### Step 4. TF-IDF approach on the dataset

In [44]:
from gensim import corpora, models
# tfidf model and corpus
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [46]:
# tfidf scores
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


### Step 5. LDA using Bag of Words

In [50]:
# Initialize the LDA model using Multicore approach from the gensim library.
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 10, id2word = dictionary, passes = 50)

KeyboardInterrupt: 

In [51]:
# For each topic explore the words occurence for that topic and their relative weights
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

NameError: name 'lda_model' is not defined

### Step 6. LDA using TF-IDF approach

In [55]:
# Initialize the LDA model using Multicore approach from the gensim library.
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 10, id2word = dictionary, passes = 50)


KeyboardInterrupt



In [56]:
# For each topic explore the words occurence for that topic and their relative weights
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

NameError: name 'lda_model_tfidf' is not defined